In [35]:
using ITensors
include("../LogLoss/structs.jl");
include("../LogLoss/encodings.jl");
using Random, Distributions
using StatsPlots

In [16]:
trial_basis = SplitBasis("Hist Split", "Stoudenmire")

SplitBasis(Hist Split Stoudenmire)

In [52]:
dataset = rand(Uniform(0, 1), (100, 5));

In [39]:
trial_basis.splitmethod(dataset, 10, 0, 1)

5-element Vector{Vector{Float64}}:
 [0.0, 0.05661754934919361, 0.16294771851398465, 0.3186470657161396, 0.38707191295606064, 0.467035778760092, 0.5870803013710604, 0.6466977501386456, 0.7553905759276793, 0.8798159627080451, 1.0]
 [0.0, 0.13977725835576282, 0.2315692229495817, 0.28764484258759604, 0.39458579050366743, 0.6009195055943646, 0.6752784914841644, 0.7691688198251883, 0.8323545266797916, 0.9250731818126827, 1.0]
 [0.0, 0.14772611774779631, 0.299545584128809, 0.4012521000500769, 0.47666041393228054, 0.5251300684517926, 0.6293204653009934, 0.796304164623922, 0.877097911786122, 0.9610277125786575, 1.0]
 [0.0, 0.05868388219035592, 0.18341936299543532, 0.23274745483566, 0.39943332393626485, 0.4340730349798505, 0.5587074782642288, 0.6631161781517494, 0.7880450990295782, 0.8711178443521234, 1.0]
 [0.0, 0.18520800619346833, 0.30205593453981494, 0.4326177918216357, 0.46733458589537163, 0.5370610473304153, 0.6180200576374533, 0.7362761113057179, 0.858867755962968, 0.9058591962473725, 1.0

In [189]:
opts2 = Options(; nsweeps=20, chi_max=20,  update_iters=1, verbosity=0, dtype=Complex{Float64}, lg_iter=KLD_iter,
bbopt=BBOpt("CustomGD"), track_cost=false, eta=0.05, rescale = (false, true), d=8, aux_basis_dim=2, encoding=SplitBasis("Hist Split", "Stoudenmire"))

Options
  nsweeps: Int64 20
  chi_max: Int64 20
  cutoff: Float64 1.0e-10
  update_iters: Int64 1
  verbosity: Int64 0
  dtype: ComplexF64 <: Number
  lg_iter: KLD_iter (function of type typeof(KLD_iter))
  bbopt: BBOpt
  track_cost: Bool false
  eta: Float64 0.05
  rescale: Tuple{Bool, Bool}
  d: Int64 8
  aux_basis_dim: Int64 2
  encoding: SplitBasis


In [190]:
s = siteinds(8, 5);

In [191]:
y = rand([0, 1], 100)

100-element Vector{Int64}:
 1
 1
 1
 0
 0
 1
 1
 0
 0
 0
 ⋮
 0
 0
 1
 0
 0
 1
 1
 1
 0

In [193]:
enc_args = opts2.encoding.init(dataset, y; opts=opts2)

2-element Vector{Vector{Any}}:
 []
 [[[0.0, 0.27834939485729887, 0.5066947370921807, 0.722440466035096, 1.0], [0.0, 0.24223656817802391, 0.4130325177894598, 0.6870340301449538, 1.0], [0.0, 0.21062580491740113, 0.3813719028494877, 0.662803990561182, 1.0], [0.0, 0.2101233377462326, 0.44383137800440925, 0.795518780760192, 1.0], [0.0, 0.21706438905822556, 0.48266632064614834, 0.7114139614449508, 1.0]], 2, Basis(Stoudenmire)]

In [165]:
out_vec = opts.encoding.encode(0.1, opts.d, 1, enc_args...)

8-element Vector{ComplexF64}:
 -0.10297916915752506 + 0.83865077342183im
 -0.06518494074249358 - 0.5308588272403406im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im

In [185]:
opts = Options(; nsweeps=20, chi_max=20,  update_iters=1, verbosity=0, dtype=Complex{Float64}, lg_iter=KLD_iter,
bbopt=BBOpt("CustomGD"), track_cost=false, eta=0.05, rescale = (false, true), d=2, aux_basis_dim=2, encoding=Basis("Stoudenmire"))

Options
  nsweeps: Int64 20
  chi_max: Int64 20
  cutoff: Float64 1.0e-10
  update_iters: Int64 1
  verbosity: Int64 0
  dtype: ComplexF64 <: Number
  lg_iter: KLD_iter (function of type typeof(KLD_iter))
  bbopt: BBOpt
  track_cost: Bool false
  eta: Float64 0.05
  rescale: Tuple{Bool, Bool}
  d: Int64 2
  aux_basis_dim: Int64 2
  encoding: Basis


In [172]:
enc_args = opts.encoding.init

In [208]:
function get_state(x::Float64, opts::Options)
    """Get the state for a time independent encoding"""
    if !opts.encoding.istimedependent    
        enc_args = []
        state = opts.encoding.encode(x, opts.d, enc_args...)
    else
        error("Expected a time independent encoding.")
    end

    return state

end
function get_state(x::Float64, opts::Options, enc_args::Vector{Vector{Any}}, 
    j::Int)
    """Get the state for a time dependent encoding"""
    if opts.encoding.istimedependent
        state = opts.encoding.encode(x, opts.d, j, enc_args...)
    else
        error("Expected a time dependent encoding.")
    end

    return state
    
end

get_state (generic function with 3 methods)

In [211]:
out = get_state(0.4, opts2, enc_args, 2)

8-element Vector{ComplexF64}:
                  0.0 + 0.0im
                  0.0 + 0.0im
 -0.04207486899682563 - 0.11192506559274748im
   -0.349353757847636 + 0.9293301011850926im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im
                  0.0 + 0.0im

In [213]:
opts.encoding.range

(0, 1)